In [1]:
# ------------------------------------------
# 📂 Importations nécessaires
# ------------------------------------------

# Bibliothèques standard Python
import os
import json
from collections import Counter

# Bibliothèques de manipulation de données
import numpy as np
import pandas as pd
from joblib import dump

# Bibliothèques de visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
    recall_score,
    roc_curve,
    auc
)

# TensorFlow et Keras
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    Callback
)
from tensorflow.keras.layers import (
    GRU,
    Dense,
    Input,
    Dropout,
    LayerNormalization,
    Flatten,
    Attention,
    MultiHeadAttention,
    GlobalAveragePooling1D,
    Lambda
)

/Users/e.x.coco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# ------------------------------------------
# 🚀 Chargement des données locales
# ------------------------------------------

# Fichiers d'entrée
train_file = "../Cleaned_data/mitbih_train_trimmed.csv"
test_file = "../Cleaned_data/mitbih_test_trimmed.csv"

In [3]:
# ------------------------------------------
# 📥 Chargement des données avec fonction
# ------------------------------------------

def load_data(file_path):
    print(f"Chargement des données depuis {file_path}...")
    df = pd.read_csv(file_path)  # Chargement du fichier CSV
    X = df.iloc[:, :-1].values  # Toutes les colonnes sauf la dernière (features)
    y = df.iloc[:, -1].astype(int).values  # Dernière colonne = labels
    X, y = shuffle(X, y, random_state=42)  # Mélange aléatoire des données
    print(f"Nombre d'échantillons : {len(X)}")
    print("Distribution des classes :", Counter(y))  # Distribution des classes
    return X, y

# Charger les données en utilisant la fonction
train_ecgs, train_labels = load_data(train_file)
test_ecgs, test_labels = load_data(test_file)

Chargement des données depuis mitbih_train_trimmed.csv...
Nombre d'échantillons : 87553
Distribution des classes : Counter({0: 72470, 4: 6431, 2: 5788, 1: 2223, 3: 641})
Chargement des données depuis mitbih_test_trimmed.csv...
Nombre d'échantillons : 21891
Distribution des classes : Counter({0: 18117, 4: 1608, 2: 1448, 1: 556, 3: 162})


In [ ]:

# ------------------------------------------
# 🔄 Augmentation des données
# ------------------------------------------

# Fonctions d'augmentation
def add_gaussian_noise(ecg, noise_level=0.01):
   noise = np.random.normal(0, noise_level, len(ecg))
   return ecg + noise

def shift_signal(ecg, shift=10):
   return np.roll(ecg, shift)

def combine_augmentations(ecg, noise_level=0.01, shift=10):
   return shift_signal(add_gaussian_noise(ecg, noise_level), shift)

def augment_class_diverse(ecgs, deficit):
   augmented_ecgs = []
   num_per_type = deficit // 3
   remaining = deficit % 3

   # Données déphasées
   for i in range(num_per_type):
       ecg = ecgs[i % len(ecgs)]
       augmented_ecgs.append(shift_signal(ecg))

   # Données bruitées
   for i in range(num_per_type):
       ecg = ecgs[i % len(ecgs)]
       augmented_ecgs.append(add_gaussian_noise(ecg))

   # Données combinées
   for i in range(num_per_type + remaining):
       ecg = ecgs[i % len(ecgs)]
       augmented_ecgs.append(combine_augmentations(ecg))

   return augmented_ecgs



In [ ]:

# ------------------------------------------
# ➗ Séparer les données par classes
# ------------------------------------------


def separate_by_class(ecgs, labels):
   classes = {i: [] for i in range(5)}
   for ecg, label in zip(ecgs, labels):
       classes[label].append(ecg)
   return classes

# Charger les données
train_file = "mitbih_train_trimmed.csv"
test_file = "mitbih_test_trimmed.csv"

train_ecgs, train_labels = load_data(train_file)
test_ecgs, test_labels = load_data(test_file)


# Séparer par classe
train_classes = separate_by_class(train_ecgs, train_labels)
test_classes = separate_by_class(test_ecgs, test_labels)

# Trouver la taille cible (max des classes malades)
target_size = max(len(train_classes[i]) for i in range(1, 5))
print(f"\nTaille cible pour chaque classe malade : {target_size}")

# Augmenter chaque classe malade
balanced_malades_ecgs = []
balanced_malades_labels = []

for i in range(1, 5):
   deficit = target_size - len(train_classes[i])
   if deficit > 0:
       augmented = augment_class_diverse(train_classes[i], deficit)
       train_classes[i].extend(augmented)

   balanced_malades_ecgs.extend(train_classes[i])
   balanced_malades_labels.extend([i] * len(train_classes[i]))
   print(f"Classe {i}: {len(train_classes[i])} échantillons")

# Mélanger les données
balanced_malades_ecgs, balanced_malades_labels = shuffle(balanced_malades_ecgs, balanced_malades_labels)

# Préparation des données de test (malades uniquement)
test_malades_ecgs = []
test_malades_labels = []
for i in range(1, 5):
   test_malades_ecgs.extend(test_classes[i])
   test_malades_labels.extend([i] * len(test_classes[i]))


# Visualisation des distributions finales
plt.figure(figsize=(15, 5))

# Distribution train
plt.subplot(1, 2, 1)
train_dist = Counter(balanced_malades_labels)
plt.pie(
   [train_dist[i] for i in range(1, 5)],
   labels=[f"Classe {i}" for i in range(1, 5)],
   autopct='%1.1f%%',
   startangle=90,
   colors=['#B8D8E8', '#C7E5D6', '#D5C8E6', '#E8D8E8']
)
plt.title("Distribution des classes malades (Train)")

# Distribution test
plt.subplot(1, 2, 2)
test_dist = Counter(test_malades_labels)
plt.pie(
   [test_dist[i] for i in range(1, 5)],
   labels=[f"Classe {i}" for i in range(1, 5)],
   autopct='%1.1f%%',
   startangle=90,
   colors=['#B8D8E8', '#C7E5D6', '#D5C8E6', '#E8D8E8']
)
plt.title("Distribution des classes malades (Test)")

plt.tight_layout()
plt.show()

In [ ]:
# Normalisation des données
scaler = StandardScaler()
normalized_train_ecgs = scaler.fit_transform(balanced_malades_ecgs)
normalized_test_ecgs = scaler.transform(test_malades_ecgs)

# Reshape pour GRU
normalized_train_ecgs = normalized_train_ecgs.reshape(normalized_train_ecgs.shape[0], normalized_train_ecgs.shape[1], 1)
normalized_test_ecgs = normalized_test_ecgs.reshape(normalized_test_ecgs.shape[0], normalized_test_ecgs.shape[1], 1)

# Convertir les labels en arrays NumPy
balanced_malades_labels = np.array(balanced_malades_labels)
test_malades_labels = np.array(test_malades_labels)

print("Shapes des données :")
print("Train:", normalized_train_ecgs.shape)
print("Test:", normalized_test_ecgs.shape)

In [ ]:
# Callback pour suivre le recall
class RecallCallback(Callback):
    def __init__(self, validation_data=None, training_data=None):
        super(RecallCallback, self).__init__()
        self.validation_data = validation_data
        self.training_data = training_data
        self.train_recalls = []
        self.val_recalls = []

    def on_epoch_end(self, epoch, logs={}):
        # Calcul du recall sur les données d'entraînement
        y_pred = self.model.predict(self.training_data[0])
        y_pred_classes = np.argmax(y_pred, axis=1)
        train_recall = recall_score(self.training_data[1], y_pred_classes, average='macro')
        self.train_recalls.append(train_recall)
        logs['recall'] = train_recall

        # Calcul du recall sur les données de validation
        if self.validation_data:
            val_pred = self.model.predict(self.validation_data[0])
            val_pred_classes = np.argmax(val_pred, axis=1)
            val_recall = recall_score(self.validation_data[1], val_pred_classes, average='macro')
            self.val_recalls.append(val_recall)
            logs['val_recall'] = val_recall

In [ ]:
def find_auto_balanced_thresholds(y_true_bin, y_pred_proba, min_recall=0.95, n_thresholds=1000):
    """
    Trouve automatiquement les seuils optimaux en équilibrant recall et précision.
    """
    n_classes = y_true_bin.shape[1]
    optimal_thresholds = []
    final_metrics = []

    for classe in range(n_classes):
        print(f"\nOptimisation pour la classe {classe + 1}:")

        # Générer les seuils à tester
        probs_class = y_pred_proba[:, classe]
        min_prob, max_prob = np.min(probs_class), np.max(probs_class)
        thresholds = np.linspace(min_prob, max_prob, n_thresholds)

        # Calculer les métriques pour tous les seuils
        all_metrics = []
        for threshold in thresholds:
            y_pred_bin = (probs_class >= threshold).astype(int)

            true_pos = np.sum((y_true_bin[:, classe] == 1) & (y_pred_bin == 1))
            false_pos = np.sum((y_true_bin[:, classe] == 0) & (y_pred_bin == 1))
            false_neg = np.sum((y_true_bin[:, classe] == 1) & (y_pred_bin == 0))

            recall = true_pos / (true_pos + false_neg + 1e-10)
            precision = true_pos / (true_pos + false_pos + 1e-10)
            f1 = 2 * (precision * recall) / (precision + recall + 1e-10)

            all_metrics.append({
                'threshold': threshold,
                'recall': recall,
                'precision': precision,
                'f1': f1
            })

        # Trouver les points d'équilibre potentiels
        valid_metrics = []
        for min_precision_test in np.linspace(0.1, 0.9, 50):
            # Filtrer les seuils qui donnent un recall suffisant
            recall_valid = [m for m in all_metrics if m['recall'] >= min_recall]
            if not recall_valid:
                continue

            # Parmi ceux-là, chercher ceux avec une précision suffisante
            precision_valid = [m for m in recall_valid if m['precision'] >= min_precision_test]
            if not precision_valid:
                continue

            # Calculer un score d'équilibre
            best_for_threshold = max(precision_valid, key=lambda x: x['f1'])
            balance_score = best_for_threshold['recall'] * best_for_threshold['precision']

            valid_metrics.append({
                'min_precision': min_precision_test,
                'metrics': best_for_threshold,
                'balance_score': balance_score
            })

        if valid_metrics:
            # Choisir le meilleur point d'équilibre
            best_balance = max(valid_metrics, key=lambda x: x['balance_score'])
            best_metric = best_balance['metrics']
            used_min_precision = best_balance['min_precision']
        else:
            # Si aucun point d'équilibre n'est trouvé, prendre le meilleur compromis
            print(f"Attention: Impossible de trouver un équilibre optimal pour la classe {classe + 1}")
            best_metric = max(all_metrics, key=lambda x: x['recall'] * x['precision'])
            used_min_precision = best_metric['precision']

        optimal_thresholds.append(best_metric['threshold'])
        final_metrics.append(best_metric)

        print(f"Classe {classe + 1}:")
        print(f"- Seuil optimal trouvé: {best_metric['threshold']:.4f}")
        print(f"- Précision minimum utilisée: {used_min_precision:.4f}")
        print(f"- Recall obtenu: {best_metric['recall']:.4f}")
        print(f"- Précision obtenue: {best_metric['precision']:.4f}")
        print(f"- F1-score: {best_metric['f1']:.4f}")

        # Visualiser la courbe precision-recall pour cette classe
        plt.figure(figsize=(10, 6))

        # Tracer la courbe precision-recall
        precisions = [m['precision'] for m in all_metrics]
        recalls = [m['recall'] for m in all_metrics]
        plt.plot(recalls, precisions, 'b-', label='Precision-Recall curve')

        # Marquer le point optimal
        plt.plot(best_metric['recall'], best_metric['precision'], 'ro',
                markersize=10, label='Point optimal')

        # Lignes de référence
        plt.axvline(x=min_recall, color='g', linestyle='--',
                   label=f'Recall minimum ({min_recall})')
        plt.axhline(y=used_min_precision, color='r', linestyle='--',
                   label=f'Précision minimum ({used_min_precision:.2f})')

        plt.xlabel('Recall')
        plt.ylabel('Précision')
        plt.title(f'Courbe Precision-Recall - Classe {classe + 1}')
        plt.grid(True)
        plt.legend()
        plt.show()

    return optimal_thresholds, final_metrics

def predict_with_optimal_thresholds(probas, thresholds):
    """
    Applique les seuils optimaux aux probabilités prédites.
    """
    n_samples = len(probas)
    predictions = np.zeros(n_samples, dtype=int)

    for i in range(n_samples):
        # Comparer chaque probabilité avec son seuil
        above_threshold = probas[i] >= thresholds

        if np.any(above_threshold):
            # Si au moins une classe dépasse son seuil,
            # choisir celle avec la plus grande marge au-dessus de son seuil
            margins = probas[i] / np.array(thresholds)
            predictions[i] = np.argmax(margins)
        else:
            # Si aucune classe ne dépasse son seuil,
            # choisir celle avec la plus grande probabilité relative
            predictions[i] = np.argmax(probas[i])

    return predictions

In [ ]:
# Paramètres du modèle
input_shape = normalized_train_ecgs.shape[1:]
gru_units = 64
dropout_rate = 0.1

def gru_attention_block(inputs, gru_units, dropout_rate):
    x = GRU(gru_units, return_sequences=True)(inputs)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = GRU(gru_units * 2, return_sequences=True)(x)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)
    attention_output = Attention()([x, x])
    return attention_output

# Construction du modèle
inputs = Input(shape=input_shape)
x = gru_attention_block(inputs, gru_units, dropout_rate)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = LayerNormalization()(x)
x = Dropout(dropout_rate)(x)
outputs = Dense(4, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Préparation des données
val_split = 0.2
split_idx = int(normalized_train_ecgs.shape[0] * (1 - val_split))
X_train = normalized_train_ecgs[:split_idx]
X_val = normalized_train_ecgs[split_idx:]
y_train = balanced_malades_labels[:split_idx] - 1
y_val = balanced_malades_labels[split_idx:] - 1

In [ ]:
# Initialisation du callback de recall
recall_callback = RecallCallback(
    validation_data=(X_val, y_val),
    training_data=(X_train, y_train)
)

# Callbacks ajustés
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=10,  # Garder à 10
        restore_best_weights=True,
        mode='min'  # Explicitement spécifier le mode
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,  # Changé de 0.5 à 0.3 pour une réduction plus progressive
        patience=6,  # Augmenté de 5 à 6
        min_lr=1e-6,
        mode='min',
        cooldown=2  # Ajouter un cooldown pour éviter des changements trop fréquents
    ),
    recall_callback
]

In [ ]:
# Entraînement
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1,
    shuffle=True  # Assurer un bon mélange des données à chaque époque
)

In [ ]:
# Sauvegarder le scaler
dump(scaler, 'ecg_scaler_m2.joblib')
print("Scaler sauvegardé sous 'ecg_scaler_m2.joblib'.")

# Sauvegarder le modèle
model.save('ecg_model_m2.h5')
print("Modèle sauvegardé sous 'ecg_model_m2.h5'.")

In [ ]:
# Visualisation des métriques d'entraînement
plt.figure(figsize=(15, 5))

# Courbe de perte
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Évolution de la fonction de perte')
plt.xlabel('Époque')
plt.ylabel('Perte')
plt.legend()
plt.grid(True)

# Courbe d'accuracy
plt.subplot(1, 3, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Évolution de l\'accuracy')
plt.xlabel('Époque')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Courbe de recall
plt.subplot(1, 3, 3)
plt.plot(recall_callback.train_recalls, label='Training Recall')
plt.plot(recall_callback.val_recalls, label='Validation Recall')
plt.title('Évolution du Recall')
plt.xlabel('Époque')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Affichage des résultats précis
print("\n--- Résultats des métriques d'entraînement et de validation ---")
print(f"Perte finale sur l'ensemble d'entraînement : {history.history['loss'][-1]:.4f}")
print(f"Perte finale sur l'ensemble de validation : {history.history['val_loss'][-1]:.4f}")
print(f"Meilleure perte (validation) : {min(history.history['val_loss']):.4f}")

print(f"\nAccuracy finale sur l'ensemble d'entraînement : {history.history['accuracy'][-1]:.4f}")
print(f"Accuracy finale sur l'ensemble de validation : {history.history['val_accuracy'][-1]:.4f}")
print(f"Meilleure accuracy (validation) : {max(history.history['val_accuracy']):.4f}")

if recall_callback.train_recalls and recall_callback.val_recalls:
    print(f"\nRecall final sur l'ensemble d'entraînement : {recall_callback.train_recalls[-1]:.4f}")
    print(f"Recall final sur l'ensemble de validation : {recall_callback.val_recalls[-1]:.4f}")
    print(f"Meilleur recall (validation) : {max(recall_callback.val_recalls):.4f}")
else:
    print("\nAucune valeur de Recall disponible.")


In [ ]:
# === Résultats avec le seuil de base (0.5) ===
print("=== Résultats avec le seuil de base (0.5) ===")

# Courbes ROC pour le seuil de base
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']

# Tracer les courbes ROC pour chaque classe avec seuil de base
for i in range(4):
    fpr, tpr, _ = roc_curve(test_labels_bin[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=colors[i], lw=2,
             label=f'Classe {i+1} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC - Seuil de base (0.5)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# Calcul des prédictions avec le seuil de base (0.5)
y_pred_classes_base = (y_pred_proba >= 0.5).astype(int).argmax(axis=1) + 1  # +1 pour les classes 1-4

# Matrices de confusion pour le seuil de base
cm_base = confusion_matrix(test_malades_labels, y_pred_classes_base)
cm_percent_base = (cm_base.astype('float') / cm_base.sum(axis=1)[:, np.newaxis] * 100)

# Matrice de confusion en effectifs
plt.figure(figsize=(10, 8))
sns.heatmap(cm_base, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'],
            yticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies classes')
plt.title('Matrice de Confusion (Effectifs) - Seuil de base (0.5)')
plt.show()

# Matrice de confusion en pourcentages
plt.figure(figsize=(10, 8))
sns.heatmap(cm_percent_base, annot=True, fmt='.1f', cmap='Blues',
            xticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'],
            yticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies classes')
plt.title('Matrice de Confusion (%) - Seuil de base (0.5)')
plt.show()

# Calcul des métriques détaillées
metrics_base = []
for i in range(4):
    true_pos = cm_base[i, i]
    false_neg = np.sum(cm_base[i, :]) - true_pos
    false_pos = np.sum(cm_base[:, i]) - true_pos
    true_neg = np.sum(cm_base) - true_pos - false_pos - false_neg

    precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) != 0 else 0
    recall = true_pos / (true_pos + false_neg) if (true_pos + false_neg) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    support = true_pos + false_neg

    metrics_base.append({
        "Classe": f"Classe {i+1}",
        "Précision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Support": support
    })

# Rapport de classification pour le seuil de base
print("\nRapport de classification complet pour le seuil de base (0.5):")
print(classification_report(test_malades_labels, y_pred_classes_base, digits=4))

# Affichage des métriques sous forme de tableau
metrics_base_df = pd.DataFrame(metrics_base)
metrics_base_df = metrics_base_df.set_index("Classe")
print("\nTableau récapitulatif des métriques pour le seuil de base:")
print(metrics_base_df)

# Visualisation des métriques sous forme de tableau
fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('tight')
ax.axis('off')
table_data = table(ax, metrics_base_df, loc='center', colWidths=[0.2]*len(metrics_base_df.columns))
table_data.auto_set_font_size(False)
table_data.set_fontsize(10)
table_data.scale(1.2, 1.2)
plt.title("Tableau récapitulatif des métriques - Seuil de base (0.5)", fontsize=14)
plt.show()


In [ ]:
# === Résultats avec les seuils optimaux ===
print("=== Résultats avec les seuils optimaux ===")

# Courbes ROC avec seuils optimaux
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']

# Tracer les courbes ROC pour chaque classe avec seuils optimaux
for i in range(4):
    fpr, tpr, thresholds = roc_curve(test_labels_bin[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)

    # Trouver les coordonnées du seuil optimal
    optimal_idx = np.argmin(np.abs(thresholds - optimal_thresholds[i]))
    optimal_fpr = fpr[optimal_idx]
    optimal_tpr = tpr[optimal_idx]

    # Tracer la courbe ROC
    plt.plot(fpr, tpr, color=colors[i], lw=2,
             label=f'Classe {i+1} (AUC = {roc_auc:.2f})')

    # Ajouter un point pour le seuil optimal
    plt.scatter(optimal_fpr, optimal_tpr, color=colors[i], s=100, edgecolors='black',
                label=f'Seuil optimal Classe {i+1} ({optimal_thresholds[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC avec seuils optimaux')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# Application des seuils optimaux
y_pred_classes_optimal = predict_with_optimal_thresholds(y_pred_proba, optimal_thresholds) + 1

# Matrices de confusion pour les seuils optimaux
cm_optimal = confusion_matrix(test_malades_labels, y_pred_classes_optimal)
cm_percent_optimal = (cm_optimal.astype('float') / cm_optimal.sum(axis=1)[:, np.newaxis] * 100)

# Matrice de confusion en effectifs
plt.figure(figsize=(10, 8))
sns.heatmap(cm_optimal, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'],
            yticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies classes')
plt.title('Matrice de Confusion (Effectifs) - Seuils optimaux')
plt.show()

# Matrice de confusion en pourcentages
plt.figure(figsize=(10, 8))
sns.heatmap(cm_percent_optimal, annot=True, fmt='.1f', cmap='Blues',
            xticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'],
            yticklabels=['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies classes')
plt.title('Matrice de Confusion (%) - Seuils optimaux')
plt.show()

# Affichage des métriques détaillées
metrics_optimal = []
for i in range(4):
    true_pos = cm_optimal[i, i]
    false_neg = np.sum(cm_optimal[i, :]) - true_pos
    false_pos = np.sum(cm_optimal[:, i]) - true_pos
    true_neg = np.sum(cm_optimal) - true_pos - false_pos - false_neg

    precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) != 0 else 0
    recall = true_pos / (true_pos + false_neg) if (true_pos + false_neg) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    support = true_pos + false_neg

    metrics_optimal.append({
        "Classe": f"Classe {i+1}",
        "Seuil Optimal": optimal_thresholds[i],
        "Précision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Support": support
    })

# Rapport de classification pour les seuils optimaux
print("\nRapport de classification complet pour les seuils optimaux:")
print(classification_report(test_malades_labels, y_pred_classes_optimal, digits=4))

# Affichage des métriques sous forme de tableau
metrics_optimal_df = pd.DataFrame(metrics_optimal)
metrics_optimal_df = metrics_optimal_df.set_index("Classe")
print("\nTableau récapitulatif des métriques pour les seuils optimaux:")
print(metrics_optimal_df)

# Visualisation des métriques sous forme de tableau
fig, ax = plt.subplots(figsize=(14, 5))
ax.axis('tight')
ax.axis('off')
table_data = table(ax, metrics_optimal_df, loc='center', colWidths=[0.2]*len(metrics_optimal_df.columns))
table_data.auto_set_font_size(False)
table_data.set_fontsize(10)
table_data.scale(1.2, 1.2)
plt.title("Tableau récapitulatif des métriques - Seuils optimaux", fontsize=14)
plt.show()
